In [1]:
import os
import json
import shutil
import pandas as pd
import numpy as np
import tensorflow as tf

2024-04-07 17:21:12.377652: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-07 17:21:12.377764: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-07 17:21:12.496440: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
df = pd.read_csv('/kaggle/input/cassava-leaf-disease-classification/train.csv')
df.head()

,image_id,label
0,1000015157.jpg,0
1,1000201771.jpg,3
2,100042118.jpg,1
3,1000723321.jpg,1
4,1000812911.jpg,3


In [3]:
import hashlib

def hash_data(df, col_name='label'):
    label = df[col_name].copy()
    data_bytes = label.astype(str).apply(lambda x: x.encode()).sum()
    return hashlib.sha256(data_bytes).hexdigest()

original_hash = hash_data(df, col_name='label')

In [4]:
original_hash

'42c3449423df232a51c9d515f78d11958f6793f01cf920a0c8ccd5025fde86ba'

In [3]:
f = open('/kaggle/input/cassava-leaf-disease-classification/label_num_to_disease_map.json')
mapping = json.load(f)
mapping

{'0': 'Cassava Bacterial Blight (CBB)',
 '1': 'Cassava Brown Streak Disease (CBSD)',
 '2': 'Cassava Green Mottle (CGM)',
 '3': 'Cassava Mosaic Disease (CMD)',
 '4': 'Healthy'}

In [6]:
y = df['label'].copy()

def label_flip(y, percentage, class_to_flip):

  # Find indices of the class to flip
  class_indices = np.where(y == class_to_flip)[0]

  # Ensure there are labels to flip for the specified class
  if len(class_indices) == 0:
    print(f"Warning: No labels found for class {class_to_flip} to flip.")
    return y

  # Calculate number of labels to flip from this class
  num_flips = int(len(class_indices) * percentage)

  # Randomly select indices from the class labels to flip
  flip_indices = np.random.choice(class_indices, num_flips, replace=False)

  # Create a copy of the labels to avoid modifying the original array
  y_flipped = y.copy()

  # Flip the labels at the selected indices
  y_flipped[flip_indices] = 1 - y_flipped[flip_indices]  # Invert the labels

  return y_flipped

percentage = .15  # 15% of labels to flip
class_to_flip = 3
flipped_labels = label_flip(y, percentage, class_to_flip)

# Make sure there are no negative values in the flipped dataset
df['label']= flipped_labels.abs()

In [7]:
altered_hash = hash_data(df, col_name='label')
altered_hash

'd2fd92d683792f092070146e97f5e4f2ce51c3e0ca922754b4b95a1e18443d37'

In [10]:
def compare_hash(original_hash, altered_hash):
    if original_hash != altered_hash:
        print("Training data has been altered")
    else:
        print("Data hasn't been touched")
        
compare_hash(original_hash, altered_hash)

Training data has been altered


In [11]:
compare_hash(original_hash, original_hash)

Data hasn't been touched
